# 🚛 Vehicle Route Optimization System

## Project Overview
This notebook implements a **Vehicle Routing Problem (VRP)** solution for optimizing fertilizer delivery from a warehouse to multiple KD (distribution) points in Lahore, Pakistan.

### Key Features:
- **Real-world routing** using OSRM (Open Source Routing Machine)
- **Mixed vehicle fleet** with different capacities and costs
- **Cost optimization** to minimize total delivery expenses
- **Interactive visualization** with route maps

### Business Goal:
Minimize transportation costs while meeting all customer demands and respecting vehicle capacity constraints.

---

# 📚 Library Imports

Import all required libraries for optimization, routing, and visualization.

In [31]:
!pip install ortools osrm folium

In [32]:
!pip install fuzzywuzzy

In [33]:
# Core libraries
import requests
import json
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# Google OR-Tools for optimization
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# Visualization
import folium

print("✅ All libraries loaded successfully!")

✅ All libraries loaded successfully!


# 📍 Data & Preprocessing

Define locations (warehouse + KDs) and their demands. The warehouse must be at index 0.

In [34]:
from fuzzywuzzy import process
import re

# --- Step 1: KD list with coordinates ---
kd_coords = {
    "KD Chak 137": ("31°34'14.3\"N", "72°53'20.5\"E"),
    "KD Chak 144": ("31°35'30.4\"N", "72°50'10.1\"E"),
    "KD Chak 151": ("31°29'42.0\"N", "72°51'31.5\"E"),
    "KD Jani Shah": ("31°39'10.8\"N", "72°51'55.0\"E"),
    "KD Mathruma": ("31°40'37.7\"N", "72°54'34.0\"E"),
    "KD Chak 224": ("31°28'49.9\"N", "72°39'10.6\"E"),
    "KD Ada Gagh Chowk": ("31°25'52.0\"N", "72°44'44.5\"E"),
    "KD Lodhia (Jand Wala Colony)": ("31°38'41.5\"N", "72°49'04.8\"E"),
    "KD Ubhaan": ("31°38'37.9\"N", "72°38'11.0\"E"),
    "KD Ada Sheikhan": ("31°36'49.1\"N", "72°27'21.0\"E"),
    "KD Kurk Mohamadi": ("31°36'49.7\"N", "72°45'00.2\"E"),
    "KD Mouza Satiana": ("31°25'53.2\"N", "72°29'46.2\"E"),
    "KD Chak No 201": ("31°23'31.4\"N", "72°46'43.5\"E"),
    "KD Chak No 206": ("31°27'16.8\"N", "72°47'53.1\"E"),
    "KD Tibbi Lalera": ("31°38'11.5\"N", "72°30'11.6\"E"),
    "KD Mangini": ("31°35'50.4\"N", "72°36'43.0\"E"),
    "KD Wara-Thatha Shah Muhammad": ("31°32'11.3\"N", "72°32'06.5\"E"),
    "KD Chak no 240-(Adda 240 Handlana)": ("31°28'58.3\"N", "72°42'19.4\"E"),
    "KD Barkhudar (Adda Kot Nijabat)": ("31°34'41.7\"N", "72°40'03.8\"E"),
    "KD Kalri": ("31°39'32.6\"N", "72°32'22.1\"E"),
}

# --- Step 2: Sales dataset ---
sales_data = [
    ("KD Ada Gagh Chowk", 117),
    ("KD Ada Sheikhan", 908),
    ("KD Barkhudar (Adda Kot Nijabat)", 118),
    ("KD Chak 137", 150),
    ("KD Chak 144", 496),
    ("KD Chak 151", 208),
    ("KD Chak 201", 78),
    ("KD Chak 206", 65),
    ("KD Chak 224", 170),
    ("KD Chak 240 (Adda 240 Handlana)", 269),
    ("KD Jani Shah", 36),
    ("KD Kalri", 20),
    ("KD Kurk Muhammadi", 86),  # spelling diff
    ("KD Lodhia (Jand Wala Colony)", 100),
    ("KD Mathruma", 142),
    ("KD Mouza Satiana", 205),
    ("KD Ubhaan", 558),
    ("KD Wara-Thatha Shah Muhammad", 83),
]

# --- Step 3: Utility to convert DMS → Decimal Degrees ---
def dms_to_dd(dms_str):
    dms_str = dms_str.strip()
    parts = re.split('[°\'"]', dms_str)
    deg, minutes, seconds, hemi = float(parts[0]), float(parts[1]), float(parts[2]), parts[3]
    dd = deg + minutes/60 + seconds/3600
    if hemi in ['S', 'W']:
        dd *= -1
    return dd

# --- Step 4: Match sales KD names to KD coords ---
all_locations = []

# crucial
#all_locations.append(("Warehouse", 31.522334, 72.576500))

all_demands = []
#all_demands.append(0)
for kd_name, qty in sales_data:
    # fuzzy match
    match, score = process.extractOne(kd_name, kd_coords.keys())
    if score > 80:  # only accept good matches
        lat_dms, lon_dms = kd_coords[match]
        lat, lon = dms_to_dd(lat_dms), dms_to_dd(lon_dms)
        all_locations.append((kd_name, lat, lon))
        all_demands.append(qty)
    else:
        print(f"⚠️ Could not match {kd_name}")

# --- Step 5: Output ---
for loc in all_locations:
    print(loc)


('KD Ada Gagh Chowk', 31.43111111111111, 72.74569444444444)
('KD Ada Sheikhan', 31.61363888888889, 72.45583333333333)
('KD Barkhudar (Adda Kot Nijabat)', 31.57825, 72.66772222222222)
('KD Chak 137', 31.57063888888889, 72.88902777777778)
('KD Chak 144', 31.591777777777775, 72.83613888888888)
('KD Chak 151', 31.495, 72.85875)
('KD Chak 201', 31.392055555555554, 72.77875)
('KD Chak 206', 31.454666666666665, 72.79808333333334)
('KD Chak 224', 31.480527777777777, 72.65294444444444)
('KD Chak 240 (Adda 240 Handlana)', 31.48286111111111, 72.70538888888889)
('KD Jani Shah', 31.653, 72.86527777777778)
('KD Kalri', 31.659055555555554, 72.53947222222222)
('KD Kurk Muhammadi', 31.613805555555558, 72.75005555555556)
('KD Lodhia (Jand Wala Colony)', 31.644861111111112, 72.818)
('KD Mathruma', 31.67713888888889, 72.90944444444445)
('KD Mouza Satiana', 31.431444444444445, 72.49616666666667)
('KD Ubhaan', 31.64386111111111, 72.6363888888889)
('KD Wara-Thatha Shah Muhammad', 31.536472222222223, 72.53513

In [35]:
# Add warehouse as the first location
warehouse_location = ("Warehouse (Bhawana)", 31.522334, 72.576500)  # Your main depot

# Prepend warehouse to locations and demands
all_locations = [warehouse_location] + all_locations
all_demands = [0] + all_demands  # Warehouse has 0 demand


In [36]:
# Data preprocessing: filter locations with demand (keep warehouse + active customers)
locations = []
demands = []

for i, (location, demand) in enumerate(zip(all_locations, all_demands)):
    if i == 0 or demand > 0:  # Keep warehouse (index 0) and customers with demand
        locations.append(location)
        demands.append(demand)

print("\n📊 Data Preprocessing Results:")
print(f"   Active locations: {len(locations)}")
print(f"   Filtered out: {len(all_locations) - len(locations)} locations")
print(f"   Total demand: {sum(demands)} bags")

print("\n📦 Location Details:")
for i, (name, _, _) in enumerate(locations):
    print(f"   {i}: {name} - {demands[i]} bags")


📊 Data Preprocessing Results:
   Active locations: 19
   Filtered out: 0 locations
   Total demand: 3809 bags

📦 Location Details:
   0: Warehouse (Bhawana) - 0 bags
   1: KD Ada Gagh Chowk - 117 bags
   2: KD Ada Sheikhan - 908 bags
   3: KD Barkhudar (Adda Kot Nijabat) - 118 bags
   4: KD Chak 137 - 150 bags
   5: KD Chak 144 - 496 bags
   6: KD Chak 151 - 208 bags
   7: KD Chak 201 - 78 bags
   8: KD Chak 206 - 65 bags
   9: KD Chak 224 - 170 bags
   10: KD Chak 240 (Adda 240 Handlana) - 269 bags
   11: KD Jani Shah - 36 bags
   12: KD Kalri - 20 bags
   13: KD Kurk Muhammadi - 86 bags
   14: KD Lodhia (Jand Wala Colony) - 100 bags
   15: KD Mathruma - 142 bags
   16: KD Mouza Satiana - 205 bags
   17: KD Ubhaan - 558 bags
   18: KD Wara-Thatha Shah Muhammad - 83 bags


In [37]:
# Vehicle fleet configuration
# Define different vehicle types with capacity and cost per km
vehicles = [
    # Large trucks - for high-volume deliveries (like the 908-bag customer)
    {"count": 4, "capacity": 1000, "cost_per_km": 100.0},

    # Medium trucks - balanced capacity and cost
    {"count": 6, "capacity": 500, "cost_per_km": 70.0},

    # Small vans - cost-efficient for small deliveries
    {"count": 3, "capacity": 200, "cost_per_km": 50.0},
]

# Calculate total fleet capacity
total_capacity = sum(v["count"] * v["capacity"] for v in vehicles)
capacity_utilization = (sum(demands) / total_capacity) * 100

print("🚛 Vehicle Fleet Configuration:")
for i, v in enumerate(vehicles, 1):
    print(f"   Type {i}: {v['count']}x {v['capacity']}-bag vehicles @ ₨{v['cost_per_km']}/km")

print(f"\n📊 Fleet Statistics:")
print(f"   Total capacity: {total_capacity} bags")
print(f"   Demand: {sum(demands)} bags")
print(f"   Capacity utilization: {capacity_utilization:.1f}%")
print(f"   Status: {'✅ Sufficient' if total_capacity >= sum(demands) else '❌ Insufficient'}")

🚛 Vehicle Fleet Configuration:
   Type 1: 4x 1000-bag vehicles @ ₨100.0/km
   Type 2: 6x 500-bag vehicles @ ₨70.0/km
   Type 3: 3x 200-bag vehicles @ ₨50.0/km

📊 Fleet Statistics:
   Total capacity: 7600 bags
   Demand: 3809 bags
   Capacity utilization: 50.1%
   Status: ✅ Sufficient


# 🗺️ Distance Matrix Generation

Calculate real-world driving distances using OSRM API with fallback to Haversine distances.

In [38]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate straight-line distance between two GPS coordinates.
    Used as fallback if OSRM API is unavailable.

    Args:
        lat1, lon1: Latitude and longitude of first point
        lat2, lon2: Latitude and longitude of second point

    Returns:
        Distance in kilometers
    """
    R = 6371.0  # Earth's radius in kilometers

    # Convert to radians
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)

    # Haversine formula
    a = (sin(dlat/2)**2 +
         cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2)
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    return R * c

def get_osrm_distance_matrix(locations):
    """
    Get real driving distances using OSRM (Open Source Routing Machine).
    Provides actual road distances instead of straight-line approximations.

    Args:
        locations: List of (name, lat, lon) tuples

    Returns:
        numpy array of distances in km, or None if failed
    """
    print(f"🌐 Requesting real driving distances for {len(locations)} locations...")

    # OSRM public server (for production, use your own server)
    OSRM_URL = "http://router.project-osrm.org"

    # Prepare coordinates (OSRM uses longitude,latitude format)
    coordinates = [[lon, lat] for name, lat, lon in locations]
    coord_string = ";".join([f"{lon},{lat}" for lon, lat in coordinates])

    # Build API request
    url = f"{OSRM_URL}/table/v1/driving/{coord_string}"
    params = {
        'sources': ';'.join([str(i) for i in range(len(locations))]),
        'destinations': ';'.join([str(i) for i in range(len(locations))]),
        'annotations': 'distance,duration'
    }

    try:
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        if data['code'] == 'Ok':
            # Convert from meters to kilometers
            distances = np.array(data['distances']) / 1000
            print(f"✅ OSRM distances retrieved successfully!")
            return distances
        else:
            print(f"❌ OSRM API error: {data['message']}")
            return None

    except Exception as e:
        print(f"❌ OSRM request failed: {e}")
        return None

def create_fallback_distance_matrix(locations):
    """
    Create distance matrix using Haversine formula with road factor.
    Used when OSRM is unavailable.

    Args:
        locations: List of (name, lat, lon) tuples

    Returns:
        numpy array of distances in km
    """
    n = len(locations)
    distances = np.zeros((n, n))
    road_factor = 1.3  # Roads are ~30% longer than straight-line

    for i in range(n):
        for j in range(n):
            if i != j:
                lat1, lon1 = locations[i][1], locations[i][2]
                lat2, lon2 = locations[j][1], locations[j][2]
                distances[i][j] = haversine_distance(lat1, lon1, lat2, lon2) * road_factor

    return distances

In [39]:
# Generate distance matrix with OSRM (preferred) or Haversine (fallback)
print("🔍 Generating distance matrix...")

# Try OSRM first
distance_matrix = get_osrm_distance_matrix(locations)

# Fallback to Haversine if OSRM fails
if distance_matrix is None:
    print("⚠️  OSRM unavailable, using Haversine distances with road factor...")
    distance_matrix = create_fallback_distance_matrix(locations)
    distance_source = "Haversine + road factor"
else:
    distance_source = "OSRM real driving distances"

print(f"✅ Distance matrix created using: {distance_source}")
print(f"📏 Matrix size: {distance_matrix.shape[0]}x{distance_matrix.shape[1]}")

# Display distance matrix
print("\n📊 Distance Matrix (km):")
print("     ", end="")
for i in range(len(locations)):
    print(f"{i:6}", end="")
print()

for i in range(len(locations)):
    print(f"{i:2}: ", end="")
    for j in range(len(locations)):
        print(f"{distance_matrix[i][j]:6.1f}", end="")
    print(f"  ({locations[i][0]})")

# Summary statistics
total_distance = np.sum(distance_matrix)
avg_distance = np.mean(distance_matrix[distance_matrix > 0])  # Exclude zeros
max_distance = np.max(distance_matrix)

print(f"\n📈 Distance Statistics:")
print(f"   Average distance: {avg_distance:.1f} km")
print(f"   Maximum distance: {max_distance:.1f} km")
print(f"   Total matrix sum: {total_distance:.1f} km")

🔍 Generating distance matrix...
🌐 Requesting real driving distances for 19 locations...
✅ OSRM distances retrieved successfully!
✅ Distance matrix created using: OSRM real driving distances
📏 Matrix size: 19x19

📊 Distance Matrix (km):
          0     1     2     3     4     5     6     7     8     9    10    11    12    13    14    15    16    17    18
 0:    0.0  24.1  33.7  10.8  37.4  30.8  32.1  29.3  31.1  15.0  17.2  32.8  26.0  19.6  26.9  36.4  17.6  20.4   5.3  (Warehouse (Bhawana))
 1:   24.1   0.0  44.0  21.1  24.7  27.3  15.7   5.5   7.2  13.9   7.0  43.1  36.3  29.9  37.2  46.7  41.6  30.7  29.4  (KD Ada Gagh Chowk)
 2:   33.8  44.1   0.0  24.2  50.7  44.1  47.3  49.4  47.1  40.0  37.2  46.1  10.2  32.9  40.2  49.7  51.3  23.6  39.1  (KD Ada Sheikhan)
 3:   10.8  21.1  24.2   0.0  26.6  20.0  24.3  26.4  24.2  17.0  14.2  22.0  16.5   8.8  16.1  25.6  28.4  10.9  16.2  (KD Barkhudar (Adda Kot Nijabat))
 4:   37.4  24.7  50.8  26.6   0.0   6.6   9.2  26.9  17.4  37.5  30.5

# ⚙️ VRP Optimization

Set up and solve the Vehicle Routing Problem using Google OR-Tools.

In [40]:
def create_vehicle_specifications(vehicles):
    """
    Convert vehicle type definitions to individual vehicle specifications.
    Each vehicle type can have multiple vehicles of the same specification.

    Args:
        vehicles: List of vehicle type dictionaries

    Returns:
        List of individual vehicle specifications
    """
    vehicle_specs = []
    vehicle_id = 1

    for vehicle_type in vehicles:
        for _ in range(vehicle_type["count"]):
            vehicle_specs.append({
                'id': vehicle_id,
                'capacity': vehicle_type["capacity"],
                'cost_per_km': vehicle_type["cost_per_km"]
            })
            vehicle_id += 1

    return vehicle_specs

def calculate_route_cost(distance_km, cost_per_km):
    """
    Calculate the total cost for a route.
    Simple formula: distance × cost_per_km

    Args:
        distance_km: Route distance in kilometers
        cost_per_km: Vehicle cost per kilometer

    Returns:
        Total route cost in currency units
    """
    return distance_km * cost_per_km

def create_cost_evaluator(cost_per_km, distance_matrix, manager):
    """
    Create cost evaluation function for OR-Tools optimizer.
    This function calculates the cost of traveling between any two points.

    Args:
        cost_per_km: Vehicle's cost per kilometer
        distance_matrix: Matrix of distances between all locations
        manager: OR-Tools routing manager

    Returns:
        Cost evaluation function
    """
    def cost_callback(from_index, to_index):
        # Convert routing indices to location indices
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)

        # Calculate cost: distance × cost_per_km
        distance_km = distance_matrix[from_node][to_node]
        cost = distance_km * cost_per_km

        # Scale by 100 for integer precision in OR-Tools
        return int(round(cost * 100))

    return cost_callback

In [41]:
def solve_vrp(locations, demands, vehicles, distance_matrix):
    """
    Solve the Vehicle Routing Problem using Google OR-Tools.

    Args:
        locations: List of location tuples (name, lat, lon)
        demands: List of demand values for each location
        vehicles: List of vehicle type specifications
        distance_matrix: Matrix of distances between locations

    Returns:
        Tuple of (solution_routes, optimization_success)
    """
    print("\n⚙️ Setting up VRP optimization...")

    # Step 1: Create individual vehicle specifications
    vehicle_specs = create_vehicle_specifications(vehicles)
    print(f"🚛 Total vehicles available: {len(vehicle_specs)}")

    # Step 2: Create OR-Tools routing model
    # Parameters: num_locations, num_vehicles, depot_index
    manager = pywrapcp.RoutingIndexManager(
        len(locations),      # Number of locations
        len(vehicle_specs),  # Number of vehicles
        0                    # Depot index (warehouse)
    )
    routing = pywrapcp.RoutingModel(manager)

    # Step 3: Register cost evaluators for each vehicle
    print("📊 Registering cost evaluators...")
    for vehicle_id, spec in enumerate(vehicle_specs):
        cost_callback = create_cost_evaluator(
            spec['cost_per_km'], distance_matrix, manager
        )
        cost_evaluator_index = routing.RegisterTransitCallback(cost_callback)
        routing.SetArcCostEvaluatorOfVehicle(cost_evaluator_index, vehicle_id)

    # Step 4: Add capacity constraints
    print("📦 Adding capacity constraints...")
    def demand_callback(from_index):
        """Return demand at the location corresponding to the index."""
        return demands[manager.IndexToNode(from_index)]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # No slack (exact capacity)
        [spec['capacity'] for spec in vehicle_specs],  # Vehicle capacities
        True,  # Start cumulative to zero
        'Capacity'
    )

    # Step 5: Configure search parameters
    print("🔍 Configuring optimization parameters...")
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.seconds = 120  # 1 minute time limit

    # Step 6: Solve the problem
    print("🎯 Solving optimization problem...")
    solution = routing.SolveWithParameters(search_parameters)

    if not solution:
        print("❌ No solution found!")
        return None, False
    # start at this level
    # Step 7: Extract routes from solution
    # CORRECTED route extraction - replace lines 370-410 in solve_vrp function

    # Step 7: Extract routes from solution
    print("✅ Solution found! Extracting routes...")
    routes = []

    for vehicle_id in range(len(vehicle_specs)):
        index = routing.Start(vehicle_id)
        route_nodes = []
        route_load = 0
        route_distance = 0

        # Follow the route path
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route_nodes.append(node)
            route_load += demands[node]

            # Calculate distance to next node
            next_index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(next_index):
                next_node = manager.IndexToNode(next_index)
                route_distance += distance_matrix[node][next_node]
            index = next_index

        # Only include routes that visit customers (more than just warehouse)
        if len(route_nodes) > 1:
            spec = vehicle_specs[vehicle_id]

            # 🔧 CRITICAL FIX: Add return trip to warehouse
            last_customer = route_nodes[-1]
            return_distance = distance_matrix[last_customer][0]  # Distance back to warehouse
            route_distance += return_distance

            # Calculate corrected cost with complete route
            route_cost = calculate_route_cost(route_distance, spec['cost_per_km'])

            routes.append({
                'vehicle_id': spec['id'],
                'route_nodes': route_nodes,
                'complete_route': route_nodes + [0],  # Show complete route including return
                'load': route_load,
                'capacity': spec['capacity'],
                'distance': route_distance,  # Now includes return trip
                'return_distance': return_distance,  # Track return leg separately
                'cost': route_cost,
                'cost_per_km': spec['cost_per_km'],
                'utilization': (route_load / spec['capacity']) * 100
            })


            # end at this

    return routes, True

In [42]:
# Execute VRP optimization
print("🚀 Starting Vehicle Route Optimization...")

solution_routes, optimization_success = solve_vrp(
    locations, demands, vehicles, distance_matrix
)

if optimization_success:
    print(f"\n🎉 Optimization completed successfully!")
    print(f"📊 Generated {len(solution_routes)} optimal routes")
else:
    print("\n❌ Optimization failed - check constraints and data")

🚀 Starting Vehicle Route Optimization...

⚙️ Setting up VRP optimization...
🚛 Total vehicles available: 13
📊 Registering cost evaluators...
📦 Adding capacity constraints...
🔍 Configuring optimization parameters...
🎯 Solving optimization problem...
✅ Solution found! Extracting routes...

🎉 Optimization completed successfully!
📊 Generated 7 optimal routes


In [52]:
# Quick validation test after optimization
if solution_routes:
    print("🔍 Warehouse Return Validation:")
    for route in solution_routes:
        complete_route = route['complete_route']
        print(f"Vehicle {route['vehicle_id']}:")
        print(f"   Route path: {complete_route}")
        print(f"   Starts at: {complete_route[0]} (warehouse)")
        print(f"   Ends at: {complete_route[-1]} (warehouse)")
        print(f"   Return distance: {route['return_distance']:.1f} km")
        print(f"   Status: {'✅ Complete round-trip' if complete_route[-1] == 0 else '❌ Missing return'}")
        print()

🔍 Warehouse Return Validation:
Vehicle 2:
   Route path: [0, 1, 7, 8, 6, 4, 11, 15, 14, 13, 0]
   Starts at: 0 (warehouse)
   Ends at: 0 (warehouse)
   Return distance: 19.6 km
   Status: ✅ Complete round-trip

Vehicle 3:
   Route path: [0, 3, 17, 0]
   Starts at: 0 (warehouse)
   Ends at: 0 (warehouse)
   Return distance: 19.6 km
   Status: ✅ Complete round-trip

Vehicle 4:
   Route path: [0, 2, 12, 0]
   Starts at: 0 (warehouse)
   Ends at: 0 (warehouse)
   Return distance: 26.1 km
   Status: ✅ Complete round-trip

Vehicle 5:
   Route path: [0, 5, 0]
   Starts at: 0 (warehouse)
   Ends at: 0 (warehouse)
   Return distance: 30.8 km
   Status: ✅ Complete round-trip

Vehicle 6:
   Route path: [0, 10, 9, 0]
   Starts at: 0 (warehouse)
   Ends at: 0 (warehouse)
   Return distance: 15.0 km
   Status: ✅ Complete round-trip

Vehicle 7:
   Route path: [0, 16, 0]
   Starts at: 0 (warehouse)
   Ends at: 0 (warehouse)
   Return distance: 17.6 km
   Status: ✅ Complete round-trip

Vehicle 11:
   R

# 📊 Results Analysis

Display and analyze the optimization results.

In [53]:
def display_optimization_results(routes, locations, demands):
    """
    Display detailed optimization results in a formatted way.

    Args:
        routes: List of optimized route dictionaries
        locations: List of location tuples
        demands: List of demand values
    """
    if not routes:
        print("❌ No routes to display!")
        return

    print("\n" + "="*70)
    print("🚛 OPTIMIZED DELIVERY ROUTES")
    print("="*70)

    total_cost = 0
    total_distance = 0
    total_load = 0

    for i, route in enumerate(routes, 1):
        # Generate route description
        route_names = [locations[node][0] for node in route['route_nodes']]
        route_path = " → ".join(route_names) + " → Warehouse"

        # Display route details
        print(f"\n🚚 Vehicle {route['vehicle_id']} ({route['capacity']} bags capacity):")
        print(f"   📍 Route: {route_path}")
        print(f"   📦 Load: {route['load']}/{route['capacity']} bags ({route['utilization']:.1f}% full)")
        print(f"   🛣️  Distance: {route['distance']:.1f} km")
        print(f"   💰 Cost: {route['distance']:.1f}km × ₨{route['cost_per_km']}/km = ₨{route['cost']:.2f}")
        print(f"   📊 Cost per bag: ₨{route['cost']/route['load']:.1f}")

        # Accumulate totals
        total_cost += route['cost']
        total_distance += route['distance']
        total_load += route['load']

    # Display summary
    print("\n" + "="*70)
    print("📊 OPTIMIZATION SUMMARY")
    print("="*70)
    print(f"🚛 Vehicles used: {len(routes)}")
    print(f"🛣️  Total distance: {total_distance:.1f} km")
    print(f"💰 Total cost: ₨{total_cost:.2f}")
    print(f"📦 Total bags delivered: {total_load}")
    print(f"📊 Average cost per bag: ₨{total_cost/total_load:.2f}")
    print(f"📊 Average cost per km: ₨{total_cost/total_distance:.2f}")

    # Efficiency metrics
    total_demand = sum(demands)
    avg_utilization = (total_load / sum(route['capacity'] for route in routes)) * 100

    print(f"\n📈 Efficiency Metrics:")
    print(f"   Demand coverage: {total_load}/{total_demand} bags ({(total_load/total_demand)*100:.1f}%)")
    print(f"   Average vehicle utilization: {avg_utilization:.1f}%")
    print(f"   Routes generated: {len(routes)} routes")

# Display results
if optimization_success and solution_routes:
    display_optimization_results(solution_routes, locations, demands)
else:
    print("❌ No optimization results to display")


🚛 OPTIMIZED DELIVERY ROUTES

🚚 Vehicle 2 (1000 bags capacity):
   📍 Route: Warehouse (Bhawana) → KD Ada Gagh Chowk → KD Chak 201 → KD Chak 206 → KD Chak 151 → KD Chak 137 → KD Jani Shah → KD Mathruma → KD Lodhia (Jand Wala Colony) → KD Kurk Muhammadi → Warehouse
   📦 Load: 982/1000 bags (98.2% full)
   🛣️  Distance: 110.4 km
   💰 Cost: 110.4km × ₨100.0/km = ₨11035.88
   📊 Cost per bag: ₨11.2

🚚 Vehicle 3 (1000 bags capacity):
   📍 Route: Warehouse (Bhawana) → KD Barkhudar (Adda Kot Nijabat) → KD Ubhaan → Warehouse
   📦 Load: 676/1000 bags (67.6% full)
   🛣️  Distance: 41.3 km
   💰 Cost: 41.3km × ₨100.0/km = ₨4130.10
   📊 Cost per bag: ₨6.1

🚚 Vehicle 4 (1000 bags capacity):
   📍 Route: Warehouse (Bhawana) → KD Ada Sheikhan → KD Kalri → Warehouse
   📦 Load: 928/1000 bags (92.8% full)
   🛣️  Distance: 70.0 km
   💰 Cost: 70.0km × ₨100.0/km = ₨6999.21
   📊 Cost per bag: ₨7.5

🚚 Vehicle 5 (500 bags capacity):
   📍 Route: Warehouse (Bhawana) → KD Chak 144 → Warehouse
   📦 Load: 496/500 bags

# 🗺️ Map Visualization & Export

Create interactive maps showing optimized routes and export results.

In [45]:
def get_osrm_route_geometry(start_coords, end_coords):
    """
    Get detailed route geometry between two points using OSRM.

    Args:
        start_coords: (latitude, longitude) of start point
        end_coords: (latitude, longitude) of end point

    Returns:
        List of [lat, lon] coordinates for the route path, or None if failed
    """
    try:
        # OSRM route API expects lon,lat format
        coord_string = f"{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}"
        url = f"http://router.project-osrm.org/route/v1/driving/{coord_string}"

        params = {
            'overview': 'full',
            'geometries': 'geojson'
        }

        response = requests.get(url, params=params, timeout=15)
        data = response.json()

        if data['code'] == 'Ok':
            route = data['routes'][0]
            geometry = route['geometry']['coordinates']
            # Convert from [lon, lat] to [lat, lon] for Folium
            path = [[coord[1], coord[0]] for coord in geometry]
            return path
        else:
            return None

    except Exception as e:
        print(f"Route geometry error: {e}")
        return None

def create_route_map(locations, routes, demands, filename="optimized_routes.html"):
    """
    Create an interactive map showing optimized delivery routes.

    Args:
        locations: List of location tuples
        routes: List of optimized route dictionaries
        demands: List of demand values
        filename: Output HTML filename

    Returns:
        Folium map object
    """


    print(f"🗺️ Creating interactive route map...")

    # Calculate map center
    center_lat = sum(loc[1] for loc in locations) / len(locations)
    center_lon = sum(loc[2] for loc in locations) / len(locations)

    # Create base map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=11,
        tiles='OpenStreetMap'
    )

    # Add warehouse marker
    folium.Marker(
        [locations[0][1], locations[0][2]],
        popup=f"🏭 {locations[0][0]}\n(Warehouse - Depot)",
        icon=folium.Icon(color='red', icon='home', prefix='fa'),
        tooltip="Warehouse (Starting Point)"
    ).add_to(m)

    # Add customer markers
    for i, (name, lat, lon) in enumerate(locations[1:], 1):
        folium.Marker(
            [lat, lon],
            popup=f"🏪 {name}\nDemand: {demands[i]} bags",
            icon=folium.Icon(color='blue', icon='shopping-cart', prefix='fa'),
            tooltip=f"{name} ({demands[i]} bags)"
        ).add_to(m)
    # Update the route path creation in create_route_map function

# Add route lines (replace the route path creation section)
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'darkblue', 'darkgreen']

    for i, route in enumerate(routes):
        color = colors[i % len(colors)]

        # Create complete route path including return to warehouse
        route_path = []
        complete_route = route.get('complete_route', route['route_nodes'] + [0])

        # Add segments for the complete route
        for j in range(len(complete_route) - 1):
            current_node = complete_route[j]
            next_node = complete_route[j + 1]

            start_coords = (locations[current_node][1], locations[current_node][2])
            end_coords = (locations[next_node][1], locations[next_node][2])

            # Get real route geometry from OSRM
            segment_path = get_osrm_route_geometry(start_coords, end_coords)

            if segment_path:
                route_path.extend(segment_path)
            else:
                # Fallback to straight line
                route_path.extend([start_coords, end_coords])

        # Add route polyline to map
        if route_path:
            folium.PolyLine(
                route_path,
                color=color,
                weight=4,
                opacity=0.8,
                popup=f"🚚 Vehicle {route['vehicle_id']}\n"
                      f"Complete Distance: {route['distance']:.1f} km\n"
                      f"Return Distance: {route.get('return_distance', 0):.1f} km\n"
                      f"Load: {route['load']} bags\n"
                      f"Cost: ₨{route['cost']:.2f}",
                tooltip=f"Vehicle {route['vehicle_id']} - Complete Route"
            ).add_to(m)


    # Add summary legend
    total_cost = sum(r['cost'] for r in routes)
    total_distance = sum(r['distance'] for r in routes)

    legend_html = f'''
    <div style="position: fixed; bottom: 50px; left: 50px; width: 350px; height: 280px;
                background-color: white; border: 2px solid grey; z-index: 9999;
                font-size: 14px; padding: 15px; border-radius: 10px;">
    <h4>🚛 Route Optimization Results</h4>
    <p><strong>🏭 Red:</strong> Warehouse (Depot)</p>
    <p><strong>🏪 Blue:</strong> Customer Locations</p>
    <p><strong>🛣️ Colored Lines:</strong> Optimized Routes</p>
    <hr>
    <p><strong>Vehicles Used:</strong> {len(routes)}</p>
    <p><strong>Total Distance:</strong> {total_distance:.1f} km</p>
    <p><strong>Total Cost:</strong> ₨{total_cost:.2f}</p>
    <p><strong>Avg Cost/Bag:</strong> ₨{total_cost/sum(demands):.1f}</p>
    <hr>
    <p><em>🌐 Powered by OSRM routing</em></p>
    </div>
    '''

    m.get_root().html.add_child(folium.Element(legend_html))

    # Save map
    m.save(filename)
    print(f"✅ Interactive map saved as '{filename}'")

    return m

In [46]:
# Create and save interactive map
if optimization_success and solution_routes:
    route_map = create_route_map(locations, solution_routes, demands)

    print("\n🎯 Map creation completed!")
    print("💡 Open 'optimized_routes.html' in your browser to view the interactive map")

    # Display map in notebook (if supported)
    try:
        route_map
    except:
        print("📱 Map display not supported in this environment")
else:
    print("❌ Cannot create map - no optimization results available")

🗺️ Creating interactive route map...
✅ Interactive map saved as 'optimized_routes.html'

🎯 Map creation completed!
💡 Open 'optimized_routes.html' in your browser to view the interactive map


# ✅ Validation & Testing

Validate the optimization results and test solution quality.

In [47]:
def validate_solution(routes, locations, demands, vehicles):
    """
    Validate the optimization solution for correctness and constraints.

    Args:
        routes: List of optimized route dictionaries
        locations: List of location tuples
        demands: List of demand values
        vehicles: List of vehicle specifications

    Returns:
        Dictionary with validation results
    """
    print("\n🔍 Validating optimization solution...")

    validation_results = {
        'valid': True,
        'errors': [],
        'warnings': [],
        'metrics': {}
    }

    if not routes:
        validation_results['valid'] = False
        validation_results['errors'].append("No routes generated")
        return validation_results

    # Test 1: Capacity constraint validation
    print("   📦 Checking capacity constraints...")
    for route in routes:
        if route['load'] > route['capacity']:
            validation_results['valid'] = False
            validation_results['errors'].append(
                f"Vehicle {route['vehicle_id']}: Load {route['load']} exceeds capacity {route['capacity']}"
            )

    # Test 2: Demand coverage validation
    print("   🎯 Checking demand coverage...")
    served_customers = set()
    total_served_demand = 0

    for route in routes:
        for node in route['route_nodes'][1:]:  # Skip warehouse
            served_customers.add(node)
            total_served_demand += demands[node]

    total_demand = sum(demands[1:])  # Exclude warehouse demand
    coverage_percentage = (total_served_demand / total_demand) * 100

    if coverage_percentage < 100:
        unserved_customers = set(range(1, len(locations))) - served_customers
        validation_results['warnings'].append(
            f"Demand coverage: {coverage_percentage:.1f}% - Unserved customers: {unserved_customers}"
        )

    # Test 3: Route validity (all routes start and end at warehouse)
    print("   🛣️  Checking route validity...")
    for route in routes:
        if route['route_nodes'][0] != 0:  # Must start at warehouse
            validation_results['valid'] = False
            validation_results['errors'].append(
                f"Vehicle {route['vehicle_id']}: Route doesn't start at warehouse"
            )

    # Test 4: Cost calculation validation
    print("   💰 Validating cost calculations...")
    for route in routes:
        expected_cost = route['distance'] * route['cost_per_km']
        if abs(route['cost'] - expected_cost) > 0.01:  # Allow small rounding errors
            validation_results['warnings'].append(
                f"Vehicle {route['vehicle_id']}: Cost calculation mismatch"
            )

    # Calculate efficiency metrics
    total_cost = sum(r['cost'] for r in routes)
    total_distance = sum(r['distance'] for r in routes)
    total_capacity_used = sum(r['capacity'] for r in routes)
    avg_utilization = (sum(r['load'] for r in routes) / total_capacity_used) * 100

    validation_results['metrics'] = {
        'total_cost': total_cost,
        'total_distance': total_distance,
        'demand_coverage': coverage_percentage,
        'avg_utilization': avg_utilization,
        'cost_per_bag': total_cost / total_served_demand if total_served_demand > 0 else 0,
        'cost_per_km': total_cost / total_distance if total_distance > 0 else 0
    }

    return validation_results

def display_validation_results(validation_results):
    """
    Display validation results in a formatted way.

    Args:
        validation_results: Dictionary with validation results
    """
    print("\n" + "="*60)
    print("✅ SOLUTION VALIDATION RESULTS")
    print("="*60)

    # Overall status
    if validation_results['valid']:
        print("🎉 Solution Status: ✅ VALID")
    else:
        print("⚠️  Solution Status: ❌ INVALID")

    # Display errors
    if validation_results['errors']:
        print("\n❌ Errors Found:")
        for error in validation_results['errors']:
            print(f"   • {error}")

    # Display warnings
    if validation_results['warnings']:
        print("\n⚠️  Warnings:")
        for warning in validation_results['warnings']:
            print(f"   • {warning}")

    # Display metrics
    if validation_results['metrics']:
        metrics = validation_results['metrics']
        print("\n📊 Solution Quality Metrics:")
        print(f"   💰 Total cost: ₨{metrics['total_cost']:.2f}")
        print(f"   🛣️  Total distance: {metrics['total_distance']:.1f} km")
        print(f"   🎯 Demand coverage: {metrics['demand_coverage']:.1f}%")
        print(f"   📦 Avg vehicle utilization: {metrics['avg_utilization']:.1f}%")
        print(f"   💵 Cost per bag: ₨{metrics['cost_per_bag']:.2f}")
        print(f"   🚗 Cost per km: ₨{metrics['cost_per_km']:.2f}")

    if not validation_results['errors'] and not validation_results['warnings']:
        print("\n🎊 Perfect solution - no issues found!")

In [48]:
# Run validation tests
if optimization_success and solution_routes:
    validation_results = validate_solution(solution_routes, locations, demands, vehicles)
    display_validation_results(validation_results)
else:
    print("❌ Cannot run validation - no optimization results available")


🔍 Validating optimization solution...
   📦 Checking capacity constraints...
   🎯 Checking demand coverage...
   🛣️  Checking route validity...
   💰 Validating cost calculations...

✅ SOLUTION VALIDATION RESULTS
🎉 Solution Status: ✅ VALID

📊 Solution Quality Metrics:
   💰 Total cost: ₨32211.03
   🛣️  Total distance: 368.2 km
   🎯 Demand coverage: 100.0%
   📦 Avg vehicle utilization: 81.0%
   💵 Cost per bag: ₨8.46
   🚗 Cost per km: ₨87.48

🎊 Perfect solution - no issues found!


In [49]:
# Performance testing and comparison
def performance_analysis(routes, distance_matrix):
    """
    Analyze solution performance and suggest improvements.

    Args:
        routes: List of optimized route dictionaries
        distance_matrix: Matrix of distances between locations
    """
    print("\n📈 Performance Analysis:")

    if not routes:
        print("   ❌ No routes to analyze")
        return

    # Utilization analysis
    utilizations = [r['utilization'] for r in routes]
    avg_utilization = sum(utilizations) / len(utilizations)
    min_utilization = min(utilizations)
    max_utilization = max(utilizations)

    print(f"   🚛 Vehicle Utilization:")
    print(f"      Average: {avg_utilization:.1f}%")
    print(f"      Range: {min_utilization:.1f}% - {max_utilization:.1f}%")

    # Route efficiency
    distances = [r['distance'] for r in routes]
    costs = [r['cost'] for r in routes]

    print(f"   🛣️  Route Efficiency:")
    print(f"      Shortest route: {min(distances):.1f} km")
    print(f"      Longest route: {max(distances):.1f} km")
    print(f"      Most expensive: ₨{max(costs):.2f}")
    print(f"      Most efficient: ₨{min(costs):.2f}")

    # Suggestions
    print(f"\n💡 Optimization Suggestions:")

    if avg_utilization < 70:
        print(f"   • Consider using fewer, larger vehicles (avg utilization: {avg_utilization:.1f}%)")

    if max_utilization - min_utilization > 30:
        print(f"   • Rebalance loads between vehicles (utilization range: {max_utilization-min_utilization:.1f}%)")

    if len(routes) > len(locations) // 2:
        print(f"   • Consider consolidating routes (using {len(routes)} vehicles for {len(locations)-1} customers)")

# Run performance analysis
if optimization_success and solution_routes:
    performance_analysis(solution_routes, distance_matrix)
else:
    print("❌ Cannot run performance analysis - no optimization results available")


📈 Performance Analysis:
   🚛 Vehicle Utilization:
      Average: 75.4%
      Range: 41.0% - 99.2%
   🛣️  Route Efficiency:
      Shortest route: 10.6 km
      Longest route: 110.4 km
      Most expensive: ₨11035.88
      Most efficient: ₨531.87

💡 Optimization Suggestions:
   • Rebalance loads between vehicles (utilization range: 58.2%)


# 🎯 Summary & Export

Final summary of results and export options.

In [50]:
# Export results to CSV (optional)
def export_results_to_csv(routes, filename="optimization_results.csv"):
    """
    Export optimization results to CSV file.

    Args:
        routes: List of optimized route dictionaries
        filename: Output CSV filename
    """
    try:
        import pandas as pd

        # Prepare data for export
        export_data = []
        for route in routes:
            export_data.append({
                'Vehicle_ID': route['vehicle_id'],
                'Capacity': route['capacity'],
                'Load': route['load'],
                'Utilization_%': route['utilization'],
                'Distance_km': route['distance'],
                'Cost_per_km': route['cost_per_km'],
                'Total_Cost': route['cost'],
                'Cost_per_bag': route['cost'] / route['load'] if route['load'] > 0 else 0,
                'Route_nodes': ', '.join(map(str, route['route_nodes']))
            })

        df = pd.DataFrame(export_data)
        df.to_csv(filename, index=False)
        print(f"✅ Results exported to '{filename}'")

    except ImportError:
        print("⚠️  Pandas not available - CSV export skipped")
    except Exception as e:
        print(f"❌ Export failed: {e}")

# Final summary
print("\n" + "="*70)
print("🎊 VEHICLE ROUTE OPTIMIZATION COMPLETED")
print("="*70)

if optimization_success and solution_routes:
    total_cost = sum(r['cost'] for r in solution_routes)
    total_distance = sum(r['distance'] for r in solution_routes)
    total_demand = sum(demands)

    print(f"✅ Optimization Status: SUCCESS")
    print(f"🚛 Vehicles deployed: {len(solution_routes)}")
    print(f"📊 Total cost: ₨{total_cost:.2f}")
    print(f"🛣️  Total distance: {total_distance:.1f} km")
    print(f"📦 Bags delivered: {sum(r['load'] for r in solution_routes)}/{total_demand}")
    print(f"💵 Average cost per bag: ₨{total_cost/total_demand:.2f}")

    print(f"\n📁 Generated Files:")
    print(f"   • optimized_routes.html (interactive map)")

    # Export CSV if requested
    export_results_to_csv(solution_routes)
    print(f"   • optimization_results.csv (data export)")

else:
    print(f"❌ Optimization Status: FAILED")
    print(f"💡 Check input data and constraints")

print(f"\n🚀 Ready for production deployment!")
print(f"📞 Contact: Route Optimization Team")
print("="*70)


🎊 VEHICLE ROUTE OPTIMIZATION COMPLETED
✅ Optimization Status: SUCCESS
🚛 Vehicles deployed: 7
📊 Total cost: ₨32211.03
🛣️  Total distance: 368.2 km
📦 Bags delivered: 3809/3809
💵 Average cost per bag: ₨8.46

📁 Generated Files:
   • optimized_routes.html (interactive map)
✅ Results exported to 'optimization_results.csv'
   • optimization_results.csv (data export)

🚀 Ready for production deployment!
📞 Contact: Route Optimization Team


In [51]:
# Quick validation test
if solution_routes:
    print("🔍 Current Route Analysis:")
    for route in solution_routes:
        route_nodes = route['route_nodes']
        print(f"Vehicle {route['vehicle_id']}:")
        print(f"   Nodes: {route_nodes}")
        print(f"   Starts at: {locations[route_nodes[0]][0]}")
        print(f"   Ends at: {locations[route_nodes[-1]][0]}")
        print(f"   Returns to warehouse: {'❓ Unknown' if len(route_nodes) < 2 else '🔍 Check needed'}")
        print()

🔍 Current Route Analysis:
Vehicle 2:
   Nodes: [0, 1, 7, 8, 6, 4, 11, 15, 14, 13]
   Starts at: Warehouse (Bhawana)
   Ends at: KD Kurk Muhammadi
   Returns to warehouse: 🔍 Check needed

Vehicle 3:
   Nodes: [0, 3, 17]
   Starts at: Warehouse (Bhawana)
   Ends at: KD Ubhaan
   Returns to warehouse: 🔍 Check needed

Vehicle 4:
   Nodes: [0, 2, 12]
   Starts at: Warehouse (Bhawana)
   Ends at: KD Kalri
   Returns to warehouse: 🔍 Check needed

Vehicle 5:
   Nodes: [0, 5]
   Starts at: Warehouse (Bhawana)
   Ends at: KD Chak 144
   Returns to warehouse: 🔍 Check needed

Vehicle 6:
   Nodes: [0, 10, 9]
   Starts at: Warehouse (Bhawana)
   Ends at: KD Chak 224
   Returns to warehouse: 🔍 Check needed

Vehicle 7:
   Nodes: [0, 16]
   Starts at: Warehouse (Bhawana)
   Ends at: KD Mouza Satiana
   Returns to warehouse: 🔍 Check needed

Vehicle 11:
   Nodes: [0, 18]
   Starts at: Warehouse (Bhawana)
   Ends at: KD Wara-Thatha Shah Muhammad
   Returns to warehouse: 🔍 Check needed

